# Build a model from scatch

This notebook shows the process of going from GIS data to an operational PRMS/pywatershed model of the Sagehen Creek Watershed, near Truckee, CA. In this notebook, we use functionality from `D-Any`, `FloPy`, and `pyGSFLOW` to construct an operational model of the basin.

This notebook does not cover in depth details of each of the functions that are used to construct the model. Instead it refers to other notebooks that cover the methods.

In [8]:
import os
import flopy
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from pathlib import Path
from dany import fill_nan_values, fill_sinks, FlowDirections, PrmsStreams
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.triangle import Triangle
from flopy.plot import styles

from dataretrieval import nwis, nldi
from pyproj import Transformer

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

Define paths for input data and model outputs 

In [4]:
data_ws = Path("../data")
output_ws = data_ws / "sagehen_voronoi_notebook"
dem_file = data_ws / "dem.img"
geospatial_ws = data_ws / "geospatial"

# soil rasters
awc = geospatial_ws / "awc.img"
ksat = geospatial_ws / "ksat.img"
clay = geospatial_ws / "clay.img"
sand = geospatial_ws / "sand.img"

# land cover rasters
nlcd2011 = geospatial_ws / "nlcd2011_imp_utm.img"
lf_cover = geospatial_ws / "us_140evc_utm.img"
lf_vegtype = geospatial_ws / "us_140evt_utm.img"

# climate rasters
ppt_rstrs = [geospatial_ws / f"climate/ppt_utm/PRISM_ppt_30yr_normal_800mM2_{i :02d}_bil.img" for i in range(1, 13)]
tmax_rstrs = [geospatial_ws / f"climate/tmax_utm/PRISM_tmax_30yr_normal_800mM2_{i :02d}_bil.img" for i in range(1, 13)]
tmin_rstrs = [geospatial_ws / f"climate/tmin_utm/PRISM_tmin_30yr_normal_800mM2_{i :02d}_bil.img" for i in range(1, 13)]

Load the Digital Elevation Model (DEM) raster

In [6]:
rstr = flopy.utils.Raster.load(dem_file)

Create a bounding box of decimal latitude and longitude for the raster.

In [11]:
epsg_utm = rstr.crs.to_epsg()
epsg_wgs84 = 4326  # decimal lat lon
crs_utm = f"EPSG:{epsg_utm}"
crs_wgs84 = f"EPSG:{epsg_wgs84}"

transformer = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True)

get decimal latitude longitude boundaries for the area of interest

In [12]:
xmin, xmax, ymin, ymax = rstr.bounds
xmin, ymin = transformer.transform(xmin, ymin)
xmax, ymax = transformer.transform(xmax, ymax)
wgs_bounds = [xmin, ymin, xmax, ymax]

Get gage information, Basin boundary, and NHDPlus stream channel for the basin using `dataretrieval`. This infomation will be used for constructing the Voronoi model grid with `FloPy`

In [14]:
info, meta = nwis.get_info(bBox=[f"{i :.2f}" for i in wgs_bounds])
info.to_crs(epsg=epsg_utm, inplace=True)
sitedf = info[info.site_no == "10343500"]
sitedf

C:\Users\jlarsen\Documents\anaconda3\envs\pyclass\Lib\site-packages\geopandas\geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
C:\Users\jlarsen\Documents\anaconda3\envs\pyclass\Lib\site-packages\geopandas\geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
C:\Users\jlarsen\Documents\anaconda3\envs\pyclass\Lib\site-packages\pandas\core\frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
C:\Users\jlarsen\Documents\anaconda3\envs\pyclass\Lib\site-packages\geopandas\geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use publ

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,geometry
2,USGS,10343500,SAGEHEN C NR TRUCKEE CA,ST,392554.0,1201413.0,39.431572,-120.237979,M,F,...,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (221299.997 4369675.311)
